In [1]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [C:\Users\ilyus\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


## NASA Mars News

In [3]:
#url for NASA's news
url ="https://redplanetscience.com/"
# open the url
browser.visit(url)

time.sleep(1)

# scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [4]:
# latest news headline and paragrapgh
news_title = soup.find_all('div', class_='content_title')[0].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text
print(news_title)
print(news_p)

Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth
Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight.


## JPL Mars Space Images—Featured Image

In [5]:
#url for Mars' image
jpl_url = 'https://spaceimages-mars.com/'

# open the jpl_url
browser.visit(jpl_url)

time.sleep(1)

# scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

# find the image
featured_image_url = soup.find_all('img')[2]["src"]

# display larger size image
print(featured_image_url)

image/mars/Icaria Fossae7.jpg


In [6]:
#create the url for the image
image = jpl_url + featured_image_url
image

'https://spaceimages-mars.com/image/mars/Icaria Fossae7.jpg'

## Mars Facts




In [7]:
#url for NASA's mars facts
facts_url = 'https://galaxyfacts-mars.com/'

# parse the url
table = pd.read_html(facts_url)
table


[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [8]:
# converting to df and renaming columns
facts = table[0]
facts.columns=['Description', 'Mars', 'Earth']
facts = facts.drop([0])
facts

,Description,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [9]:
# converting to html
fact_table = facts.to_html()
fact_table.replace('\n','')
print(fact_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature:</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


## Mars Hemispheres

In [10]:
#url for Mars' Hemispheres
hems_url = 'https://marshemispheres.com/'

# open the jpl_url
browser.visit(hems_url)

time.sleep(1)

# scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [11]:
# read to html and extract image urls
hems_data = soup.find_all("div", class_="item")

hemisphere_img_urls = []

# loop through the data to find title and url info
for image in hems_data:
    
    title = image.find("h3").text
    
    img_url = image.a["href"]
    
    url = hems_url + img_url
    
    # use requests to get full images url 
    response = requests.get(url)
    
    # create soup object
    soup = bs(response.text,"html.parser")
    
    # find full image url
    new_url = soup.find("img", class_="wide-image")["src"]
    
    # create full image url
    full_url = "https://marshemispheres.com/" + new_url
    
   
    # create a dictionary and append to the list before the loop
    hemisphere_img_urls.append({"title": title, "img_url": full_url})

hemisphere_img_urls    

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]